# Getting started
**This is a copy of [https://pystan.readthedocs.io/en/latest/getting_started.html](https://pystan.readthedocs.io/en/latest/getting_started.html).**

*[PyStan](https://pystan.readthedocs.io/en/latest/) is the Python interface for Stan and already installed in this binder environment (see [../binder/requirements.txt](../binder/requirements.txt)).*

## Using PyStan
The module’s name is `pystan` so we load the module as follows:

In [ ]:
import pystan

### Example 1: Eight Schools

The *eight schools* example appears in Section 5.5 of Gelman et al. (2003), which studied coaching effects from eight schools.

In [ ]:
schools_code = """
data {
    int<lower=0> J; // number of schools
    vector[J] y; // estimated treatment effects
    vector<lower=0>[J] sigma; // s.e. of effect estimates
}
parameters {
    real mu;
    real<lower=0> tau;
    vector[J] eta;
}
transformed parameters {
    vector[J] theta;
    theta = mu + tau * eta;
}
model {
    eta ~ normal(0, 1);
    y ~ normal(theta, sigma);
}
"""

schools_dat = {'J': 8,
               'y': [28,  8, -3,  7, -1,  1, 18, 12],
               'sigma': [15, 10, 16, 11,  9, 11, 10, 18]}

sm = pystan.StanModel(model_code=schools_code)
fit = sm.sampling(data=schools_dat, iter=1000, chains=4)

In this model, we let `theta` be transformed parameters of `mu` and `eta` instead of directly declaring theta as parameters. By parameterizing this way, the sampler will run more efficiently. In PyStan, we can also specify the Stan model using a file. For example, we can download the file 8schools.stan into our working directory and use the following call to `stan` instead:

In [ ]:
sm = pystan.StanModel(file='8schools.stan')
fit = sm.sampling(data=schools_dat, iter=1000, chains=4)

Once a model is compiled, we can use the StanModel object multiple times. This saves us time compiling the C++ code for the model. For example, if we want to sample more iterations, we proceed as follows:

In [ ]:
fit2 = sm.sampling(data=schools_dat, iter=10000, chains=4)

The object `fit`, returned from function `stan` stores samples from the posterior distribution. The `fit` object has a number of methods, including `plot` and `extract`. We can also print the `fit` object and receive a summary of the posterior samples as well as the log-posterior (which has the name `lp__`).

The method `extract` extracts samples into a dictionary of arrays for parameters of interest, or just an array.

In [ ]:
la = fit.extract(permuted=True)  # return a dictionary of arrays
mu = la['mu']

## return an array of three dimensions: iterations, chains, parameters
a = fit.extract(permuted=False)

In [ ]:
print(fit)

If `matplotlib` and `scipy` are installed, a visual summary may also be displayed using the plot() method.

In [ ]:
fit.plot()